In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
import getpass, os
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


JOB_DESCRIPTION = "System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie."


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [29]:

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
)

In [30]:
# 1. Zapytanie użytkownika

THINGS_TO_CONSIDER = [
  "1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?",
  "2. W jakim formacie chcesz otrzymać odpowiedź?",
  "3. Jaki okres czasu chcesz uwzględnić?",
]

USER_INPUT = "U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów."
prompt = (
  SystemMessage(content=JOB_DESCRIPTION) +
  SystemMessage(content=f"Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: {THINGS_TO_CONSIDER}") +
  HumanMessage(content=USER_INPUT) +
  AIMessage(content='Na czym Ci najbardziej zależy?')

).format_messages()

response1 = llm.invoke(prompt)
prompt.append(response1)
prompt

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [31]:
prompt.append(HumanMessage(content='Tytuł, czas trwania, gatunek. Chciałbym, aby formatem był JSON, lubie ten format. Zaproponuj strukturę'))
response = llm.invoke(prompt)
prompt.append(response)
prompt

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [32]:
# prompt = prompt[:4]
prompt

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [33]:
URL = "https://helios.pl/wroclaw/kino-helios-aleja-bielany/repertuar"

from langchain_core.documents import Document
from typing import List
from langchain.document_loaders import WebBaseLoader
from langchain_core.tools import tool

GOOD = False

@tool
def scrape_data_from_web() -> str:
  """
  This tool scrapes data from the web
  """
  loader = WebBaseLoader(URL)
  print('sraka')
  loader = loader.load()
  return loader[0].page_content

# @tool 
# def 

tools = [ scrape_data_from_web ]

llm_with_tools = llm.bind_tools(tools)

def validate_output(output):
  pass

counter = 0
receipe = []
while True:
  counter += 1

  response = llm_with_tools.invoke("Wybierz pierwszy krok, który należy wykonać, aby stworzyć API. Wykorzystaj narzędzie")
  prompt.append(response)

  if response.tool_cals:
    receipe.append(response.tool_cals)

  if validate_output(response):
    break
  if counter > 1:
    break

prompt

# 1 wymagania api
# 

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [35]:
prompt[-1].tool_calls[0]


{'name': 'scrape_data_from_web',
 'args': {},
 'id': 'call_kWzI5z81GAQ9atE1tFXnufvn',
 'type': 'tool_call'}

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

splits = RecursiveCharacterTextSplitter().split_documents(page)

vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

USER_INPUT = (
  """
Użytkownik chce stworzyć narzędzie umożliwiające znajdywanie aktualnie wyświetlanych filmów w kinach Helios. Dodatkowo, chce uzyskać oceny tych filmów z serwisów takich jak IMDb i Rotten Tomatoes oraz mieć prosty dostęp do trailerów. Użytkownik oczekuje, że dla każdego filmu zostaną zwrócone takie informacje jak: tytuł, czas trwania i gatunek.
  """
)
relevant_docs = retriever.get_relevant_documents(USER_INPUT)
relevant_docs

[Document(metadata={'description': 'Serwis internetowy sieci kin Helios', 'language': 'pl', 'source': 'https://helios.pl/wroclaw/kino-helios-aleja-bielany/repertuar', 'title': 'Repertuar: Kino Wrocław Helios Aleja Bielany | Helios'}, page_content='Kontakt z kinem\n  \n        Repertuar\n       \n        Cennik\n       Dzieci  Wydarzenia  \n        Aktualności\n       Więcej     Repertuar - Wrocław Helios Aleja Bielany \n          Teraz gramy\n         Repertuar na kolejny tydzień w każdy wtorek po godzinie 17:00 \n    Wybierz interesujący Cię dzień\n    Dzisiaj 28 wrzJutro 29 wrzpon. 30 wrzwt. 1 paźśr. 2 paźczw. 3 paźpt. 4 paźsob. 5 paźndz. 6 paźwt. 8 paźśr. 9 paźsob. 12 paźśr. 16 paźsob. 19 paźśr. 23 paźsob. 16 lissob. 30 lisndz. 29 gru   Zobacz ile możesz zyskać!     \n      Napisy\n    \n      Dubbing\n         \n      Film polski\n        Gatunek \n        Pokaż wszystkie\n       \n        Pokaż wszystkie\n      \n        Animowany\n      \n        Komedia\n      \n        Dramat\n

In [28]:
# from langchain import hub
# prompt = hub.pull("rlm/rag-prompt")

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | StrOutputParser()
# )

# # USER_INPUT = "U: Jakie filmy są aktualnie grane w kinie Helios?"

# rag_chain.invoke("Znajdź filmy")


/home/jakubner/anaconda3/lib/python3.11/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=ChatPromptValue(messages=... response_metadata={})]), input_type=ChatPromptValue]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type